In [1]:
!pip install mediapipe

In [2]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  # handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    # handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    # cv2.putText(annotated_image, f"{handedness[0].category_name}",
    #             (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
    #             FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [3]:
import cv2

In [4]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

def get_detection_result(base_options, options, detector, img_name):
    # STEP 3: Load the input image.
    image = mp.Image.create_from_file(img_name)
    
    # STEP 4: Detect hand landmarks from the input image.
    detection_result = detector.detect(image)
    return detection_result, image


def calculate_score(detection_result, score, num):
    for handedness in detection_result.handedness:
        score += handedness[0].score
        num += 1
    return score, num

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
import os

In [6]:
directory_path = 'frames'

In [7]:
# Get a list of all files and directories in the specified path
contents = os.listdir(directory_path)
print(contents)

['Alergia', 'alta', 'baixa', 'Bem', 'cinco', 'Corona_Virus', 'Dengue', 'diabetes', 'doente', 'Dois', 'Emagrecer', 'engordar', 'eu', 'Exame', 'febre', 'fezes', 'Fisioterapia', 'frio', 'Gripe', 'Hospital', 'medico', 'nao', 'Normal', 'nove', 'oi', 'oito', 'pressao', 'problema', 'quatro', 'quente', 'remedio', 'Sangue', 'seis', 'sentir_mal', 'sete', 'Sim', 'Suor', 'Tonto', 'tosse', 'Três', 'um', 'urina', 'voce']


In [8]:
# Iterate through the sorted directories

scores_dict = {}

for directory in contents:
    print(f"Directory: {directory}")
    directory_path2 = os.path.join(directory_path, directory)

    # List files within the directory
    directory_files = os.listdir(directory_path2)

    os.makedirs(f"frames_processed/{directory}", exist_ok=True)

    score = 0
    num = 0

    # Print the file names within the directory
    for file in directory_files:
        fullpath = f"{directory_path2}/{file}"
        # STEP 5: Process the classification result. In this case, visualize it.
        detection_result, image = get_detection_result(base_options, options, detector, fullpath)
        annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
        cv2.imwrite(f"frames_processed/{directory}/{file}", cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

        score, num = calculate_score(detection_result, score, num)

    scores_dict[directory] = score / num

Directory: Alergia
Directory: alta
Directory: baixa
Directory: Bem
Directory: cinco
Directory: Corona_Virus
Directory: Dengue
Directory: diabetes
Directory: doente
Directory: Dois
Directory: Emagrecer
Directory: engordar
Directory: eu
Directory: Exame
Directory: febre
Directory: fezes
Directory: Fisioterapia
Directory: frio
Directory: Gripe
Directory: Hospital
Directory: medico
Directory: nao
Directory: Normal
Directory: nove
Directory: oi
Directory: oito
Directory: pressao
Directory: problema
Directory: quatro
Directory: quente
Directory: remedio
Directory: Sangue
Directory: seis
Directory: sentir_mal
Directory: sete
Directory: Sim
Directory: Suor
Directory: Tonto
Directory: tosse
Directory: Três
Directory: um
Directory: urina
Directory: voce
